# Model comparison

In [1]:
import gurobipy as gb
from gurobipy import GRB
import pandas as pd
import numpy as np
import math
from collections import namedtuple

## Manage datas

Read the 1000 6x6 instances

In [24]:
networks_df = pd.read_csv("data/d_it_ij_6x6_1000it.csv",
                          decimal=",",
                          header=[0,1],
                          index_col=0)

# Convert column names from str to int and start counting nodes from 0
starting_nodes = networks_df.columns.get_level_values(0).astype(int) - 1
ending_nodes = networks_df.columns.get_level_values(1).astype(int) - 1
networks_df.columns = [starting_nodes, ending_nodes]

# print(networks_df.dtypes)
networks_df.head()

0                   1                             2             \
           6         7         6         7         8         7         8    
it1  1.612014  1.454776  0.376347  1.263488  1.675461  0.383242  1.325470   
it2  0.345666  1.997958  1.022579  1.344409  0.335500  0.768256  0.724642   
it3  1.059043  1.866680  1.672658  0.410748  1.204280  1.351971  1.502285   
it4  0.297746  1.873162  1.144040  1.749892  0.975469  0.168817  0.553882   
it5  1.296527  1.255480  1.777435  1.812928  1.990680  0.990032  0.467010   

                     3             ...        26                  27  \
           9         8         9   ...        32        33        32   
it1  1.502733  0.959483  1.690435  ...  0.199034  1.911133  1.343548   
it2  1.146472  0.726154  0.654525  ...  0.126858  0.794601  1.432874   
it3  0.093011  0.149816  0.665625  ...  1.773551  0.360475  0.016188   
it4  1.758605  1.824679  0.606143  ...  1.398726  1.794652  0.178435   
it5  0.629513  0.962954  0.695574  ...  0.499949  0.563903  0.976726   

                               28                            29            
           33        34        33        34        35        34        35  
it1  0.238743  1.667782  1.273160  0.289610  1.523664  1.968861  1.483620  
it2  0.123810  1.866517  1.411055  1.125656  1.531357  0.725753  0.617501  
it3  1.866432  1.783597  1.205805  0.093973  1.818958  1.471756  1.272506  
it4  0.930916  0.497418  1.837166  1.315127  0.259269  1.973621  1.269510  
it5  1.407947  0.320083  0.168943  0.432240  0.193984  1.377742  0.847393  

[5 rows x 80 columns]

Find the nodes

In [60]:
def get_nodes(networks_df):
    starting_nodes = networks_df.columns.get_level_values(0)
    ending_nodes = networks_df.columns.get_level_values(1)
    return starting_nodes.union(ending_nodes).unique().tolist()

nodes = get_nodes(networks_df)

Functions that gives one by one the network instaces of the dataframe in the form of a list of arcs 

In [95]:
Arc = namedtuple("Arc", ["i", "j", "d_ij"])

def sixXsix_1000_networks(networks_df):
    for it in networks_df.index:  # each network instance...
        yield [Arc(i,
                   j,
                   networks_df.loc[it, (i, j)]) for i, j in networks_df.columns]  # ... has a set of arcs

## Set up different problems

Functions to build optimization problems 

In [5]:
def compute_flow(X, node, arcs, agent):
    flow_out = gb.quicksum(X[node, arc.j, agent] for arc in arcs if arc.i==node)
    flow_in = gb.quicksum(X[arc.i, node, agent] for arc in arcs if arc.j==node)
    return flow_out - flow_in

In [52]:
def set_MSPP(nodes, arcs, agents_sources, agents_terminus):

    #TODO: Try to change X[i,j,k] to X[arc,k]

    agents = range(len(agents_sources))
    MSPP_pb = gb.Model()
    MSPP_pb.setParam("OutputFlag", 0)

    # Decision variables
    X_var_shape = len(nodes), len(nodes), len(agents)
    X = MSPP_pb.addMVar(X_var_shape,
                        vtype=GRB.BINARY,  # 5) Binary constraints
                        name="X")

    # 1-3) Objective
    distance_obj = gb.quicksum(
        arc.d_ij*X[arc.i, arc.j, k]
            for arc in arcs for k in agents
    )
    MSPP_pb.setObjectiveN(distance_obj, index=0, weight=1, name="Distance")

    # 4) Flow constraints
    #TODO: Get rid of constr names
    for k in agents:
        for i in nodes:
            if i == agents_sources[k]:
                MSPP_pb.addConstr(compute_flow(X, i, arcs, k) == 1,
                                name=f"Flow constr related to agent {k} in node {i}")
            elif i == agents_terminus[k]:
                MSPP_pb.addConstr(compute_flow(X, i, arcs, k) == -1,
                                name=f"Flow constr related to agent {k} in node {i}")
            else:
                MSPP_pb.addConstr(compute_flow(X, i, arcs, k) == 0,
                                name=f"Flow constr related to agent {k} in node {i}")

    return MSPP_pb, X

In [7]:
def set_ABP(nodes, arcs, agents_sources, agents_terminus):

    agents = range(len(agents_sources))
    MSPP_PD_ABP_pb, X = set_MSPP(nodes, arcs, agents_sources, agents_terminus)

    # Additional decision variables
    Psi_var_shape = len(nodes), len(nodes)
    Psi = MSPP_PD_ABP_pb.addMVar(Psi_var_shape,
                             vtype=GRB.BINARY,  # 8) Binary constraints
                             name="Psi")
    
    # 6) Additional objective
    penalty_obj = gb.quicksum(
        Psi[arc.i, arc.j] for arc in arcs
    )
    MSPP_PD_ABP_pb.setObjectiveN(penalty_obj, index=1, weight=1, name="Penalty")

    # 7) Additonal constraints
    for arc in arcs:
        MSPP_PD_ABP_pb.addConstr(
            1/len(agents) * (gb.quicksum(X[arc.i, arc.j, k] for k in agents) - 1)
            <= Psi[arc.i, arc.j]
        )

    return MSPP_PD_ABP_pb, X, Psi

In [8]:
def set_NBP(nodes, arcs, agents_sources, agents_terminus):

    agents = range(len(agents_sources))
    MSPP_PD_NBP_pb, X = set_MSPP(nodes, arcs, agents_sources, agents_terminus)

    # Additional decision variables
    R_var_shape = len(nodes), len(agents)
    R = MSPP_PD_NBP_pb.addMVar(R_var_shape,
                           vtype=GRB.BINARY,  # 13) Binary constraints
                           name="R")
    Xi_var_shape = len(nodes)
    Xi = MSPP_PD_NBP_pb.addMVar(Xi_var_shape,
                                vtype=GRB.BINARY,  # 14) Binary constraints
                                name="Xi")

    # 9) Additional objective
    penalty_obj = gb.quicksum(
        Xi[i] for i in nodes
    )
    MSPP_PD_NBP_pb.setObjectiveN(penalty_obj, index=1, weight=1, name="Penalty")


    # 10,11) Turning on r_i constraints
    for k in agents:
        for node in nodes:
            MSPP_PD_NBP_pb.addConstr(
                R[node, k] >= gb.quicksum(X[node, arc.j, k]
                                       for arc in arcs if arc.i == node)
            )
            MSPP_PD_NBP_pb.addConstr(
                R[node, k] >= gb.quicksum(X[arc.i, node, k]
                                       for arc in arcs if arc.j == node)
            )

    # 12) Turning on xi_i constraints
    # ! Seems weird, the -1 should be outside the summation 
    for node in nodes:
        MSPP_PD_NBP_pb.addConstr(
            1/len(agents) * (gb.quicksum(R[node, k] - 1
                                         for k in agents))
        )

    return MSPP_PD_NBP_pb, X, R, Xi

In [17]:
def set_problem(problem_type, nodes, arcs, agents_sources, agents_terminus):
    params = nodes, arcs, agents_sources, agents_terminus
    if problem_type == "MSPP":
        return set_MSPP(*params)
    elif problem_type == "ABP":
        return set_ABP(*params)
    elif problem_type == "NBP":
        return set_NBP(*params)

## Solve different scenarios

Define variables

In [94]:
num_istances = 5
# problem_types = ["MSPP","ABP", "NBP", "ALP", "NLP", "AQP", "NQP"]
problem_types = ["MSPP","ABP"]
pb_idx = dict(zip(problem_types, range(len(problem_types))))

# scenarios = [3, 6, 9, 12]
scenarios = [3, 6]
num_agents_idx = dict(zip(scenarios, range(len(scenarios))))

agents_sources = [0, 1, 2, 3, 4, 5, 0, 2, 4, 1, 3, 5]
agent_to_source = dict(zip(range(12), agents_sources))

agents_terminus = [node + 30 for node in agents_sources]
agent_to_terminus = dict(zip(range(12), agents_terminus))

tot_opt_distances = np.zeros(shape=(len(problem_types), len(scenarios), num_istances))
agent_opt_distances = np.full(shape=(len(problem_types), len(scenarios), 12, num_istances),
                              fill_value=np.nan)

Useful functions to analyze problems and their results 

In [10]:
def evaluate_pb_distance_and_penalty(problem):

    #? Shold I check optimality
    #* See: https://www.gurobi.com/documentation/9.5/refman/working_with_multiple_obje.html

    problem.params.SolutionNumber = 0  # Set best solution found
    opt_solution = []

    #? Should I use distance_obj_idx = 0 and penalty_obj_index = 1
    # Add to opt_solution the value of each objective
    for obj in range(problem.NumObj):
        problem.params.ObjNumber = obj
        opt_solution.append(problem.ObjNVal)

    return opt_solution


def evaluate_agent_distance(arcs, X_values, agent):
    # ? Should I assert that nodes in arcs == nodes in X_values
    return sum( arc.d_ij for arc in arcs if math.isclose(X_values[arc.i, arc.j, agent], 1) )

Solve each case

In [91]:
for pb_type in problem_types:  # for each problem...
    for num_of_agents in scenarios:  # ...and each scenario... 
        agents = range(num_of_agents)
        for it_i, network_arcs in enumerate(sixXsix_1000_networks(networks_df.head(num_istances))):  # ...solve it for a particular instance
            pb, X, *_ = set_problem(pb_type, nodes, network_arcs,
                        [agent_to_source.get(k) for k in agents],
                        [agent_to_terminus.get(k) for k in agents])
            pb.optimize()
            tot_opt_distance, *_ = evaluate_pb_distance_and_penalty(pb)
            tot_opt_distances[pb_idx.get(pb_type),
                             num_agents_idx.get(num_of_agents),
                             it_i] = tot_opt_distance
            
            for k in agents:
                agent_opt_distances[pb_idx.get(pb_type),
                                    num_agents_idx.get(num_of_agents),
                                    k,
                                    it_i] = evaluate_agent_distance(network_arcs, X.x, k)
            
            